# StabilityAIのVLMモデルを動かす

npaka氏あざす
- https://note.com/npaka/n/n3d51dbd8a681

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import requests
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import torch
from transformers import BitsAndBytesConfig
from transformers import pipeline
import openai
from openai import OpenAI
from src.utils import download_image_from_s3

import torch
from transformers import AutoTokenizer, AutoModelForVision2Seq, AutoImageProcessor

from PIL import Image
import requests

model_id = "/media/sj-archimedes/data/03_pretrained_model/llm/stabilityai/japanese-stable-vlm"

In [ ]:
model_kwargs = {"trust_remote_code": True, "low_cpu_mem_usage": True}
model_kwargs["variant"] = "fp16"
model_kwargs["torch_dtype"] = torch.float16
model_kwargs["device_map"] = "auto"

In [ ]:
model = AutoModelForVision2Seq.from_pretrained(model_id, **model_kwargs)
processor = AutoImageProcessor.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = model.eval()

In [ ]:
df = pd.read_pickle('/media/sj-archimedes/data/share/OddAI_Library_practice/data08/lap_trainval_20221001-20231031.pkl')
df = df.query('creative_type == "image"')
df['creative_media_url'] = df['creative_media_url'].map(lambda x: x[0])
df['creative_media_hash'] = df['creative_media_hash'].map(lambda x: x[0])

In [ ]:
s3_url = df['creative_media_url'].sample(3).tolist()
print(s3_url)
image_list = [download_image_from_s3(path) for path in s3_url]

plt.subplots(1,3, figsize=(15,5))
for i, img in enumerate(image_list):
    plt.subplot(1,3,i+1)
    plt.imshow(img)
plt.show()

In [ ]:
# プロンプトの準備
prompt = """
以下は、タスクを説明する指示と、文脈のある入力の組み合わせです。要求を適切に満たす応答を書きなさい。

### 指示: 
画像を詳細に述べてください。

### 応答: 
"""

for img in image_list:
    plt.figure(figsize=(5,5))
    plt.imshow(img)
    plt.show()
    # 入力の準備
    inputs = processor(images=img, return_tensors="pt")
    text_encoding = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")
    inputs.update(text_encoding)
    
    # 推論の実行
    outputs = model.generate(
        **inputs.to(device=model.device),
        do_sample=True,
        num_beams=1,
        max_new_tokens=512,
        min_length=1,
        repetition_penalty=1.5,
    )
    generated_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0].strip()
    print(generated_text)